In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import models

In [ ]:
def load_pretrained_encoder(checkpoint_path, backbone='resnet50'):
    """Load pretrained encoder for downstream tasks"""
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    # Initialize encoder
    if backbone == 'resnet50':
        encoder = models.resnet50(pretrained=False)
        encoder.fc = nn.Identity()
    elif backbone == 'resnet18':
        encoder = models.resnet18(pretrained=False)
        encoder.fc = nn.Identity()
    
    # Load weights
    encoder.load_state_dict(checkpoint['encoder_state_dict'])
    
    return encoder

def create_linear_classifier(encoder, num_classes, freeze_encoder=True):
    """Create a linear classifier on top of the pretrained encoder"""
    if freeze_encoder:
        for param in encoder.parameters():
            param.requires_grad = False
    
    # Get encoder output dimension
    if 'resnet50' in encoder:
        encoder_dim = 2048
    elif 'resnet18' in encoder:
        encoder_dim = 512
        print("ResNet 18 initialized")
    else:
        encoder_dim = 512  # default
    
    classifier = nn.Sequential(
        encoder,
        nn.Linear(encoder_dim, num_classes)
    )
    
    return classifier

In [3]:
class ProjectionMLP(nn.Module):
    """Projection MLP for SimSiam"""
    def __init__(self, input_dim=2048, hidden_dim=2048, output_dim=2048):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(hidden_dim, output_dim),
            nn.BatchNorm1d(output_dim, affine=False)  # No bias/scale in final BN
        )
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

class PredictionMLP(nn.Module):
    """Prediction MLP for SimSiam"""
    def __init__(self, input_dim=2048, hidden_dim=512, output_dim=2048):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

class SimSiam(nn.Module):
    """
    SimSiam model implementation
    """
    def __init__(self, backbone='resnet50', proj_dim=2048, pred_dim=512):
        super().__init__()
        
        # Backbone encoder
        if backbone == 'resnet50':
            self.encoder = models.resnet50(pretrained=False)
            self.encoder.fc = nn.Identity()  # Remove classification head
            encoder_dim = 2048
        elif backbone == 'resnet18':
            self.encoder = models.resnet18(pretrained=False)
            self.encoder.fc = nn.Identity()
            encoder_dim = 512
        else:
            raise ValueError(f"Backbone {backbone} not supported")
        
        # Projection head
        self.projector = ProjectionMLP(encoder_dim, proj_dim, proj_dim)
        
        # Prediction head
        self.predictor = PredictionMLP(proj_dim, pred_dim, proj_dim)
    
    def forward(self, x1, x2):
        # Encode both views
        z1 = self.projector(self.encoder(x1))
        z2 = self.projector(self.encoder(x2))
        
        # Predict
        p1 = self.predictor(z1)
        p2 = self.predictor(z2)
        
        return p1, p2, z1.detach(), z2.detach()

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimSiam(backbone='resnet18').to(device)

/home/asharab/miniconda3/envs/ssl_project/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asharab/miniconda3/envs/ssl_project/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
# --- Load pretrained encoder ---
encoder = load_pretrained_encoder('/home/asharab/Documents/Masters/SSL_PROJECT/Model/checkpoints_new_backbone/simsiam_encoder.pth', backbone='resnet18')
model = create_linear_classifier(encoder, num_classes=100, freeze_encoder=True).to('cuda')

# --- CIFAR-100 Dataloaders ---
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),          # Random horizontal flip
    transforms.RandomCrop(224, padding=4),           # Random crop with padding
    transforms.ColorJitter(brightness=0.2, 
                           contrast=0.2, 
                           saturation=0.2, 
                           hue=0.1),                   # Random color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

train_set = datasets.CIFAR100(root='./data', train=True, transform=transform, download=False)
test_set = datasets.CIFAR100(root='./data', train=False, transform=transform, download=False)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)

# --- Training setup ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model[-1].parameters(), lr=0.001, weight_decay=1e-4)
#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

/home/asharab/miniconda3/envs/ssl_project/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asharab/miniconda3/envs/ssl_project/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
model

In [7]:
from torch.utils.tensorboard import SummaryWriter
import torch
from tqdm import tqdm

writer = SummaryWriter(log_dir='./runs/linear_eval_experiment')

# --- Train only linear head ---
for epoch in range(100):
    model.train()
    correct, total, total_loss = 0, 0, 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1:03d}", leave=False)
    for x, y in pbar:
        x, y = x.to('cuda'), y.to('cuda')

        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

        pbar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Acc': f"{100 * correct / total:.2f}%"
        })

    #scheduler.step()
    acc = 100 * correct / total
    writer.add_scalar('Train/Loss', total_loss / len(train_loader), epoch + 1)
    writer.add_scalar('Train/Accuracy', acc, epoch + 1)

    print(f"Epoch {epoch+1:3d} | Loss: {total_loss:.4f} | Train Acc: {acc:.2f}%")

# --- Evaluation on Test Set ---
model.eval()
correct, total = 0, 0

pbar = tqdm(test_loader, desc="Evaluating", leave=False)
with torch.no_grad():
    for x, y in pbar:
        x, y = x.to('cuda'), y.to('cuda')
        preds = model(x).argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

test_acc = 100 * correct / total
writer.add_scalar('Test/Accuracy', test_acc, 100)

print(f"\nFinal Linear Evaluation Accuracy on CIFAR-100: {test_acc:.2f}%")

# --- Save final model weights ---
torch.save(model.state_dict(), 'linear_head_final_weights.pth')
print("Final model weights saved as 'linear_head_final_weights.pth'")

writer.close()

Epoch   1 | Loss: 1564.8314 | Train Acc: 10.25%


Epoch   2 | Loss: 1525.6230 | Train Acc: 12.11%


Epoch   3 | Loss: 1495.9007 | Train Acc: 13.37%


Epoch   4 | Loss: 1472.7487 | Train Acc: 14.21%


Epoch   5 | Loss: 1454.9549 | Train Acc: 15.45%


...

Epoch  97 | Loss: 1211.8219 | Train Acc: 26.55%


Epoch  98 | Loss: 1208.5280 | Train Acc: 26.82%


Epoch  99 | Loss: 1210.8917 | Train Acc: 26.40%


Epoch 100 | Loss: 1210.1002 | Train Acc: 26.71%



Final Linear Evaluation Accuracy on CIFAR-100: 24.62%
Final model weights saved as 'linear_head_final_weights.pth'
